<img src="img/python-logo-notext.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;"><b>Object orientation part 1: Classes</b></div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>

## Properties

How can we create class for points that allows us to access its position both via`x` and
`y` coordinates, as well as using radius and angle?

In [ ]:
import math


class GeoPointV0:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def get_radius(self):
        return math.hypot(self.x, self.y)

    def get_angle(self):
        return math.atan2(self.y, self.x)

    def __repr__(self):
        return f"GeoPointV0({self.x:.1f}, {self.y:.1f}, r={self.get_radius():.2f}, θ={self.get_angle():.2f})"

In [ ]:
p = GeoPointV0()
p

In [ ]:
assert p.x == 0.0
assert p.y == 0.0
assert p.get_radius() == 0.0
assert p.get_angle() == 0.0

In [ ]:
p = GeoPointV0(1.0, 0.0)
p

In [ ]:
assert p.x == 1.0
assert p.y == 0.0
assert p.get_radius() == 1.0
assert p.get_angle() == 0.0

In [ ]:
p = GeoPointV0(0.0, 2.0)
p

In [ ]:
from math import isclose, pi

assert p.x == 0.0
assert p.y == 2.0
assert p.get_radius() == 2.0
assert isclose(p.get_angle(), pi / 2)

It is inconvenient that the attributes `x`
and `y` of `GeoPointV0` must be treated differently than `radius` and `angle`:

In [ ]:
p = GeoPointV0(1.0, 1.0)
print(p.x, p.y, p.get_radius(), p.get_angle())

In [ ]:
import math


class GeoPointV1:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    @property
    def radius(self):
        return (self.x**2 + self.y**2) ** 0.5

    @property
    def angle(self):
        return math.atan2(self.y, self.x)

    def __repr__(self):
        return f"GeoPointV1({self.x:.1f}, {self.y:.1f}, r={self.radius:.2f}, θ={self.angle:.2f})"

In [ ]:
p = GeoPointV1()
p

In [ ]:
assert p.x == 0.0
assert p.y == 0.0
assert p.radius == 0.0
assert p.angle == 0.0

In [ ]:
p = GeoPointV1(1.0, 0.0)
p

In [ ]:
assert p.x == 1.0
assert p.y == 0.0
assert p.radius == 1.0
assert p.angle == 0.0

In [ ]:
p = GeoPointV1(0.0, 2.0)
p

In [ ]:
from math import isclose, pi

assert p.x == 0.0
assert p.y == 2.0
assert p.radius == 2.0
assert isclose(p.angle, pi / 2)

In [ ]:
GeoPointV1(1.0, 0.0)

In [ ]:
GeoPointV1(0.0, 2.0)

In [ ]:
p = GeoPointV1(1.0, 1.0)
print(p.x, p.y, p.radius, p.angle)

## Setters for properties:

Properties can also be modified:

In [ ]:
import math


class GeoPointV2:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    @property
    def radius(self):
        return (self.x**2 + self.y**2) ** 0.5

    @radius.setter
    def radius(self, new_radius):
        old_radius = self.radius
        # Check for `old_radius == 0`...
        self.x *= new_radius / old_radius
        self.y *= new_radius / old_radius

    @property
    def angle(self):
        return math.atan2(self.y, self.x)

    def __repr__(self):
        return f"GeoPointV1({self.x:.1f}, {self.y:.1f}, r={self.radius:.2f}, θ={self.angle:.2f})"

In [ ]:
p = GeoPointV2(3.0, 4.0)
print("Original point:  ", p)
p.radius = 10.0
print("Set radius to 10:", p)

In [ ]:
assert p.radius == 10.0

## Class methods and factories

A factory is a function (or class) used to build object instances. Class
methods are a powerful feature of Python that is useful for implementing
factories.

Class methods are methods typically called on a class (as opposed to an
object). In contrast to static methods (which receive no information about the
class they are called on) they are passed a class object as argument that can
be used to perform operations that depend on the class (such as creating
instances).

In [ ]:
from dataclasses import dataclass

In [ ]:
@dataclass
class Color:
    r: float = 0.0
    g: float = 0.0
    b: float = 0.0
    color_table = {
        "white": (1.0, 1.0, 1.0),
        "red": (1.0, 0.0, 0.0),
        "green": (0.0, 1.0, 0.0),
        "blue": (0.0, 0.0, 1.0)
    }

    @classmethod
    def from_string(cls, color):
        return cls(*cls.color_table.get(color, (0.0, 0.0, 0.0)))
    
    @classmethod
    def from_unsigned(cls, r, g, b):
        return cls(r/255, g/255, b/255)

In [ ]:
Color(0.5, 0.5, 0.5)

In [ ]:
Color.from_string("red")

In [ ]:
Color.from_unsigned(255, 0, 0)


If the constructor arguments of a subclass are compatible with the superclass,
the class methods of the superclass can directly be used as factories for the
subclass.

In [ ]:
@dataclass
class AlphaColor(Color):
    alpha: float = 1.0

In [ ]:
AlphaColor(0.5, 0.5, 0.5)

In [ ]:
AlphaColor.from_string("red")

In [ ]:
AlphaColor.from_unsigned(255, 0, 0)

## Attributes of classes

Most attributes are defined at the instance level, i.e.,
each object has its own values for the attributes. But sometimes it
makes sense to define attributes on the class level as well:

In [ ]:
class CountedAdder:
    # Attribut der Klasse, wird von allen Instanzen geteilt
    num_counters = 0

    def __init__(self, value):
        CountedAdder.num_counters += 1
        # Instanzvariable (-attribut): Jede Instanz hat eigene Werte dafür
        self.value = value

    def describe(self):
        print(
            f"One of {CountedAdder.num_counters} adders. "
            f"This one adds {self.value} to its argument."
        )

    def add(self, n):
        return self.value + n

In [ ]:
print(CountedAdder.num_counters)
a1 = CountedAdder(10)
print(CountedAdder.num_counters)
a2 = CountedAdder(20)
print(CountedAdder.num_counters)

In [ ]:
print(a1.add(1))
print(a2.add(2))

In [ ]:
a1.describe()
a2.describe()

In [ ]:
print(CountedAdder.num_counters)
print(a1.num_counters)
print(a2.num_counters)

In [ ]:
print(CountedAdder.add)
print(a1.add)
print(a2.add)

### Inheritance

In [ ]:
class LoggingAdder(CountedAdder):
    def add(self, n):
        print(f"Adding {self.value} to {n}")
        return self.value + n

In [ ]:
a3 = LoggingAdder(30)
print(a3.add(3))
print(a3.num_counters)

In [ ]:
a1.describe()
a2.describe()
a3.describe()

In [ ]:
# Method Resolution Order:
LoggingAdder.mro()

In [ ]:
print(CountedAdder.add)
print(a1.add)
print(a2.add)
print(LoggingAdder.add)
print(a3.add)

In [ ]:
print(CountedAdder.add)
print(a1.add.__func__)
print(a2.add.__func__)
print(LoggingAdder.add)
print(a3.add.__func__)

In [ ]:
a1.__dict__["value"] = 15

In [ ]:
a1.add(0)

In [ ]:
LoggingAdder.__dict__

## For experts: access to attributes

Python allows us to modify access to attributes in several places.

## Attributes of classes

 When accessing `C.name`, Python does the following:

 - If `name` is a key in `C.__dict__`:
   - `v=C.__dict__['name']`
   - If `v` is a descriptor (i.e., `type(v).__get__` is defined:
     - Result is `type(v).__get__(v, None, C)`
   - If `v` is not a descriptor:
     - Result is `v`
 - If `name` is not a key in `C.__dict__`:
   - The base classes of `C` are traversed in Method Resolution Order and
     this procedure is performed for each class

## Attributes of instances

 When accessing `object.name`, Python does the following:

 - If `name` is an overriding descriptor `v` in `C` or one of the
   Base classes of `C` is (`type(v)` has methods `__get__()` and
   `__set__()`:
   - The result is `type(v).__get__(v, object, C)`
 - Else, if `name` is a key in `object.__dict__` :
   - The result is `object.__dict__['name']`
 - Otherwise, `object.name` delegates the search to the class, as above
   described
   - If this finds a descriptor `v`, then the result is
     `type(v).__get__(v, object, C)`
   - If a value `v` is found that is not a descriptor, then `v`
     returned
 - If no value is found and `C.__getattr__` is defined, then
   `C.__getattr__(object, 'name')` is called to get the value
 - Otherwise an `AttributeError` exception will be thrown

 This process can be overridden by the `__getattribute__` method.

In [ ]:
class LoggingDescriptor:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, owner):
        print(f"__get__({self}, {instance}, {owner})")
        print(f"  __dict__ == {instance.__dict__}")
        return instance.__dict__.get(self.name, "nothing")

In [ ]:
class OverridingLoggingDescriptor(LoggingDescriptor):
    def __set__(self, instance, value):
        print(f"__set__({self}, {instance}, {value}")
        instance.__dict__[self.name] = value

In [ ]:
class YourClass:
    f = LoggingDescriptor("f")
    g = OverridingLoggingDescriptor("g")

In [ ]:
yc = YourClass()
print(yc.f, yc.g)

In [ ]:
yc.f = 234
yc.g = 345

In [ ]:
print(yc.f, yc.g)

In [ ]:
class MyClass:
    def g(self, x):
        print(self, x)


def f(x, y):
    print(x, y)

In [ ]:
mc = MyClass()
print(mc.__class__)

In [ ]:
print(MyClass.g)
print(mc.g.__qualname__)
print(mc.g.__get__)

In [ ]:
print(f.__get__)

In [ ]:
bound_f = f.__get__(mc, MyClass)
bound_g = mc.g
print(bound_f)
print(bound_g)

In [ ]:
bound_f(3)
bound_g(3)
mc.g(3)